In [ ]:
import boto3
import re
import os
import numpy as np
import pandas as pd
import sagemaker
from time import gmtime, strftime

In [ ]:
role = 'arn:aws:iam::638608113287:role/service-role/AmazonSageMaker-ExecutionRole-20180731T132167'
sess = sagemaker.Session()

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [ ]:
image = '638608113287.dkr.ecr.us-east-1.amazonaws.com/faster-rcnn:gpu'
instance_type = 'ml.p2.16xlarge'
instance_count = 1
output_path="s3://model-artifacts-alkymi/faster-rcnn/"
data_location = "s3://training-data-alkymi/pageseg/20190226"

hyperparameters = {
  "batch_size": "64",
  "epochs":"2",
  "lr": "0.0001",
  "lr_decay_gamma": "0.1",
  "lr_patience": "2",
  "patience": "4",
  "imdb_name": "pdfpages",
  "num_workers": "16",
  "USE_FLIPPED": "False"
}

In [ ]:
estimator = sagemaker.estimator.Estimator(image,
                                          role, instance_count, instance_type,
                                          output_path=output_path,
                                          sagemaker_session=sess, 
                                          hyperparameters=hyperparameters)

estimator.fit(data_location)